In [1]:
import os
!git clone https://github.com/ibrahim-elsawy/konafa
# !pip install -r /content/konafa/colab_req.txt
!pip install datasets
#!pip install torch==1.7.1
!pip install transformers==4.3.0rc1
!pip install sentencepiece 
!pip install catalyst==20.6

Cloning into 'konafa'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 164 (delta 96), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (164/164), 75.74 KiB | 8.42 MiB/s, done.
Resolving deltas: 100% (96/96), done.
     |████████████████████████████████| 225kB 7.5MB/s 
     |████████████████████████████████| 245kB 13.0MB/s 
     |████████████████████████████████| 112kB 14.0MB/s 
     |████████████████████████████████| 2.8MB 7.6MB/s 
     |████████████████████████████████| 901kB 47.9MB/s 
     |████████████████████████████████| 3.2MB 53.7MB/s 
     |████████████████████████████████| 1.2MB 7.8MB/s 
     |████████████████████████████████| 368kB 7.9MB/s 
     |████████████████████████████████| 122kB 35.4MB/s 
     |████████████████████████████████| 163kB 26.5MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 


In [2]:
os.chdir('/content/konafa')
import torch
import transformers
from catalyst import dl
from src.runners import DistilMLMRunner
# from src.models import DistilpegasusStudentModel, PegasusForMLM #BertForMLM  DistilbertStudentModel
from catalyst.core import MetricAggregationCallback
from torch.utils.data import DataLoader
from src.callbacks import (
    CosineLossCallback,
    KLDivLossCallback,
    MaskedLanguageModelCallback,
    MSELossCallback,
    PerplexityMetricCallbackDistillation,
    SmoothingLossCallback
)
import pandas as pd
from typing import Iterable, Union

# import torch
from torch.utils.data import Dataset
from tqdm.auto import tqdm
# import transformers
from transformers import AutoTokenizer
from transformers.data.data_collator import DataCollatorForLanguageModeling
import numpy as np
from transformers.data.data_collator import default_data_collator
from torch import nn
from typing import Dict, List
from datasets import load_dataset
import datasets
from datasets import Dataset

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/local/lib/python3.7/dist-packages/yaml/constructor.py:126: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 

In [3]:
import os 
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive")

Mounted at /content/drive


In [4]:
train_dataset = datasets.load_from_disk("./xsumtrainDataset_1")
valid_dataset = datasets.load_from_disk("./xsumvalidDataset_1")
teacher = torch.load('teacher_model.pt', map_location=torch.device('cuda')).to('cuda')
student = torch.load('st_3dec_3enc_2.pt', map_location=torch.device('cuda')).to('cuda')
# student = torch.load('trained_student.pt', map_location=torch.device('cuda')).to('cuda')
########################### freezing the emb layers:
for param in student.model.shared.parameters():
  param.requires_grad = False
for param in student.model.encoder.embed_tokens.parameters():
  param.requires_grad = False
for param in student.model.encoder.embed_positions.parameters():
  param.requires_grad = False
for param in student.model.decoder.embed_tokens.parameters():
  param.requires_grad = False
for param in student.model.decoder.embed_positions.parameters():
  param.requires_grad = False
  
model = torch.nn.ModuleDict({"teacher": teacher, "student": student})

In [5]:
#train_dataset.set_format('torch', columns=['input_ids', 'attention_mask'], device='cuda')
#valid_dataset.set_format('torch', columns=['input_ids', 'attention_mask'], device='cuda')
train_data = Dataset.from_dict(train_dataset[0 : 50000])
valid_data = Dataset.from_dict(valid_dataset[0 : 5000])

train_data.set_format('torch', columns=['input_ids', 'attention_mask','decode_ids','decode_mask'], device='cuda')
valid_data.set_format('torch', columns=['input_ids', 'attention_mask','decode_ids','decode_mask'], device='cuda')

In [6]:
train_dataloader = DataLoader(
    #train_dataset['train']
    train_data, batch_size=16
)
valid_dataloader = DataLoader(
    valid_data, batch_size=16
)
loaders = {"train": train_dataloader, "valid": valid_dataloader}
# del train_dataset
# del valid_dataset
#teacher = torch.load('teacher_model.pt', map_location=torch.device('cuda')).to('cuda')
#student = torch.load('st_3dec_3enc.pt', map_location=torch.device('cuda')).to('cuda')
#model = torch.nn.ModuleDict({"teacher": teacher, "student": student})

In [7]:
callbacks = {
    # "masked_lm_loss": MaskedLanguageModelCallback(),
    "mse_loss": MSELossCallback(),
    "cosine_loss": CosineLossCallback(),
    "kl_div_loss": KLDivLossCallback(),
    "label_smooth_loss": SmoothingLossCallback(),
    "loss": MetricAggregationCallback(
        prefix="loss",
        mode="weighted_sum",
        metrics={
            "cosine_loss": 1.0,
            # "masked_lm_loss": 1.0,
            "kl_div_loss": 1.0,
            "mse_loss": 1.0,
            "label_smooth_loss": 1.0,
        }
    ),
    "optimizer": dl.OptimizerCallback(),
    # "perplexity": PerplexityMetricCallbackDistillation()
}

In [8]:
runner = DistilMLMRunner(device=torch.device("cuda"))
runner.device = torch.device("cuda")
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
runner.train(
    model=model,
    optimizer=optimizer, 
    loaders=loaders,
    verbose=True,
    #check=True,
    callbacks=callbacks,
    num_epochs = 3,
    #check = True,
    # engine='cpu',
)

1/3 * Epoch (train):   0% 0/3125 [00:00<?, ?it/s]

KeyError: ignored

In [ ]:
torch.save(model['student'],'trained_student2.pt')

In [13]:
callbacks['loss'].metrics.items()

dict_items([('cosine_loss', 1.0), ('kl_div_loss', 1.0), ('mse_loss', 1.0), ('label_smooth_loss', 1.0)])